In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(':memory:')

In [4]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [5]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [6]:
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address
    FROM Customers
    WHERE Country = 'Germany' or Country = 'France' or City = 'Madrid'
    LIMIT 10
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [23]:
pd.read_sql(
    '''
    SELECT Country, Count(Country) as Cnt 
    FROM Customers 
    GROUP BY Country
    ORDER BY Cnt DESC
    LIMIT 3
    ''',
    con,
)

,Country,Cnt
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [37]:
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate
    FROM Shippers 
    left join Orders on Orders.ShipperID = Shippers.ShipperID
    ORDER BY OrderID ASC
    LIMIT 9,1
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,16.07.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [7]:
pd.read_sql(
    '''
    SELECT ProductName, Price
    FROM Products
    join OrderDetails on Products.ProductID = OrderDetails.ProductID
    where OrderID =
        (
        SELECT a.OrderID 
        FROM (
            SELECT *, sum(Quantity*Price) as OrdPrs FROM OrderDetails
            join Products on OrderDetails.ProductID = Products.ProductID
            group by OrderID
            order by OrdPrs DESC
            limit 1
            ) a
        )
    '''
    ,con
)

,ProductName,Price
0,Sir Rodney's Marmalade,81.0
1,Côte de Blaye,263.5
2,Camembert Pierrot,34.0
3,Mozzarella di Giovanni,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [257]:
pd.read_sql(
    '''
    SELECT OrderID, ProductName, Quantity
    FROM Products
    join OrderDetails
    on Products.ProductID = OrderDetails.ProductID
    where ProductName in
            (
                select a.ProductName as ProductName
                from
                    (SELECT ProductName,sum(Quantity) as Quantity
                    FROM Products
                    join OrderDetails
                    on Products.ProductID = OrderDetails.ProductID
                    group by ProductName
                    order by Quantity desc
                    limit 1) a
            )
    group by OrderID
    order by Quantity desc
    ''',
    con,
)

,OrderID,ProductName,Quantity
0,10359,Gorgonzola Telino,70
1,10390,Gorgonzola Telino,60
2,10342,Gorgonzola Telino,56
3,10376,Gorgonzola Telino,42
4,10272,Gorgonzola Telino,40
5,10393,Gorgonzola Telino,32
6,10374,Gorgonzola Telino,30
7,10356,Gorgonzola Telino,30
8,10335,Gorgonzola Telino,25
9,10363,Gorgonzola Telino,20


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [8]:
pd.read_sql(
    '''
    SELECT SupplierName, Country, ContactName, Phone
    FROM Suppliers
    where SupplierID in 
        (
            select a.SupplierID
            from
            (SELECT SupplierID, count(OrderID) as Sum_Orders
            FROM Products
            JOIN OrderDetails
            on Products.ProductID = OrderDetails.ProductID
            group by SupplierID
            order by Sum_Orders desc
            limit 5) as a
        )
    ''', 
    con, 
)

,SupplierName,Country,ContactName,Phone
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
1,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
2,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
3,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323
4,Norske Meierier,Norway,Beate Vileid,(0)2-953010


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [9]:
pd.read_sql(
    '''
    SELECT Country, CategoryName, sum(Price*Quantity) as 'sum_by_orders'

    FROM Products
    join OrderDetails
        on Products.ProductID = OrderDetails.ProductID
    join Categories
        on Products.CategoryID = Categories.CategoryID
    join Orders
        on Orders.OrderID = OrderDetails.OrderID
    join Customers
        on Customers.CustomerID = Orders.CustomerID
    
    where Country = 'Brazil'
    group by Country, CategoryName
    order by sum_by_orders desc
    limit 1
    ''',
    con,
)

,Country,CategoryName,sum_by_orders
0,Brazil,Beverages,13690.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [262]:
pd.read_sql(
    '''
    SELECT Country, max(sum_order_price) as max_Order, min(sum_order_price) as min_Order,
    max(sum_order_price) - min(sum_order_price) as diff
    FROM
        (
        select Country, sum(Price*Quantity) as sum_order_price
        from Customers a
            join Orders b
                on a.CustomerID = b.CustomerID 
            join OrderDetails c 
                on b.OrderID = c.OrderID 
            join Products d
                on c.ProductID = d.ProductID 
            
            where Country = 'USA'
            group by c.OrderID
            --order by sum_order_price desc
          )   
    ''',
    con,
)

,Country,max_Order,min_Order,diff
0,USA,7698.45,60.0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [220]:
pd.read_sql(
    '''
    select (FirstName || " " ||  LastName) as 'Full_Name', count(OrderID) as 'sum_orders', 2024 - substring(BirthDate,7,4) as age
    FROM Employees a
    join Orders b
    on a.EmployeeID = b.EmployeeID
    group by FirstName
    order by age asc
    limit 3
    ''',
    con,
)

,Full_Name,sum_orders,age
0,Anne Dodsworth,6,55
1,Nancy Davolio,29,56
2,Janet Leverling,31,61


10. Сколько банок крабового мяса всего было заказано.

In [102]:
pd.read_sql(
    '''
    SELECT ProductName, sum(Quantity*Unit) as 'Quantity'
    FROM OrderDetails a
    join Products b
    on a.ProductID = b.ProductID
    where ProductName like '%Crab%'
    ''',
    con,
)

,ProductName,Quantity
0,Boston Crab Meat,6144
